In [1]:
import DWM1_Tokenizer
import DWM2_Global_Token_Replace_NewDict
import DWM3_Entropy_Cluster
# Settings for Tokenizer
inputSampleName = r'S3'
delimiter=','
hasHeader = True
tokenizerType = 'Splitter'
removeDuplicateTokens = True

# Setting for global replacement DWM2
runReplacement = True
minFreqStdToken = 5
minLenStdToken = 3
maxFreqErrToken = 3

# Cluster Settings


DWM1_Tokenizer.tokenizeCycle(inputSampleName, delimiter, hasHeader, removeDuplicateTokens, tokenizerType)
prefixString = inputSampleName +','+tokenizerType+','+str(removeDuplicateTokens)
fileToCluster = inputSampleName + '-Tokenized.txt'
if runReplacement:
    DWM2_Global_Token_Replace_NewDict.globalReplace(inputSampleName, minFreqStdToken, minLenStdToken, maxFreqErrToken)
    fileToCluster = inputSampleName + '-TokenReplace.txt'
    prefixString = prefixString+','+str(minFreqStdToken)+','+str(minLenStdToken)+','+str(maxFreqErrToken)
else:
    prefixString = prefixString+',0,0,0'

# Cluster fixed settings
epsilonIterate = 0.0
runClusterMetrics = False
runFinalMetrics = True
createFinalJoin = False

Tokenizing References
Total References= 868
Total Tokens Found = 11157
Total Tokens Output = 10861
Dictionary Load = 67077
<class 'dict'>
Global Replace Cycle
Total References= 868
Total Tokens= 10861
Unique Tokens= 2908
Minimum Frequency of Standard Token =  5
Minimum Length of Standard Token =  3
Maximum Frequency of Error Token =  3
References Processed= 868
Sorted Token Size = 2908
Clean Token Size = 1298
*Stop Replacements here
References Processed =  868
Total Replacement Pairs = 21
Tokens Read = 10861
Tokens Changed =  29
References Changed = 28


In [2]:
status="done"

In [3]:
print(status)

done


In [ ]:
# # Cluster Robot Settings
# muStart = 0.60
# muEnd = 0.60
# muIncr = 0.05
# betaStart = 14
# betaEnd = 14
# betaIncr = 1
# sigmaStart = 145
# sigmaEnd = 145
# sigmaIncr = 1
# epsilonStart = 35
# epsilonEnd = 35
# epsilonIncr = 1
# mu = muStart
# while mu <= muEnd:
#     beta = betaStart
#     while beta <= betaEnd:
#         sigma = sigmaStart
#         while sigma <= sigmaEnd:
#             epsilon = epsilonStart
#             while epsilon <= epsilonEnd:
#                 suffixString =','+str(epsilon)+','+str(epsilonIterate)
#                 resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, beta, sigma, mu, muIncr,epsilon,  \
#                                                      epsilonIterate,runClusterMetrics, runFinalMetrics, createFinalJoin, hasHeader)
#                 seg0a = ','+str(resultsList[0])+','+str(resultsList[1])+','+str(resultsList[2])+','+str(resultsList[3])
#                 seg0b =','+str(resultsList[4])+','+str(resultsList[5])+','+str(resultsList[6])
#                 seg1 = ','+str(beta)+','+str(sigma)+','+str(mu)+ ',' +str(muIncr)
#                 seg2 = ','+str(resultsList[7])+','+str(resultsList[8])+','+str(resultsList[9])
#                 outString = prefixString+seg0a+seg0b+suffixString+seg1+seg2+'\n'
#                 resultsFile = open('RobotResults.txt','a+')
#                 resultsFile.write(outString)
#                 resultsFile.close()
#                 epsilon = epsilon + epsilonIncr
#             sigma = sigma + sigmaIncr
#         beta = beta + betaIncr
#     mu = mu + muIncr


 

## Bayesian Optimization of Hyperparameters

In [4]:
import numpy as np

Create a function of the parameters that returns the F score

In [5]:
def entropy_F1(beta=14,sigma=145,mu=0.6,muIncr=0.05,epsilon=35,epsilonIterate=0):
    resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, 
                                  beta, 
                                  sigma, 
                                  mu, 
                                  muIncr,
                                  epsilon,
                                  epsilonIterate,
                                  runClusterMetrics, 
                                  runFinalMetrics, 
                                  createFinalJoin, 
                                  hasHeader)
    return resultsList[9]

In [6]:
entropy_F1(mu=0.5,muIncr=0.05,beta=33,sigma=449,epsilon=31.8,epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.5
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 1103
NewMatrix Iterateblocks Total Time = 1.3060520393773913


>>>>>>>>>>>>
Starting Iteration mu= 0.55
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 839
NewMatrix Iterateblocks Total Time = 1.0669218823313713


>>>>>>>>>>>>
Starting Iteration mu= 0.6000000000000001
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 597
NewMatrix Iterateblocks Total Time = 0.8754348075017333


>>>>>>>>>>>>
Starting Iteration mu= 0.6500000000000001
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 277
NewMatrix Iterateblocks Total Time = 0.8119944371283054


>>>>>>>>>>>>
Starting Iteration mu= 0.7000000000000002
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!


0.14637392

Create functions for variable transformation.

In [7]:
#Softplus to transform real domain into positive
def softplus(x):
    return np.log(1+np.exp(x))

def inv_softplus(x):
    return np.log(np.exp(x)-1)

In [8]:
inv_softplus(1)

0.541324854612918

In [10]:
#Logit to transform real domain into 0-1
def logit(x):    
    return np.log(x+1e-54)-np.log(1-x+1e-54)

def inv_logit(x):
    res=np.exp(x)/(np.exp(x)+1)-2*1e-54
    if res<1e-54:
        res=0
    return res

Import BO library:

In [11]:
from bayes_opt import BayesianOptimization

Create dictionary bounds for the parameters:

In [12]:
import os, sys

In [13]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [14]:
with HiddenPrints():
    print("This will not be printed")

print("This will be printed as before")

This will be printed as before


In [15]:
#Take parameters in transformed space and return f score in transformed space
def entropy_F1_trx(beta_sp,sigma_sp,mu_logit,muIncr_logit,epsilon_sp,epsilonIterate_logit=logit(0)):
    beta_pos=int(softplus(beta_sp))
    sigma_pos=int(softplus(sigma_sp))
    mu_01=inv_logit(mu_logit)
    muIncr_01=inv_logit(muIncr_logit)
    epsilon_pos=softplus(epsilon_sp)
    epsilonIterate_01=inv_logit(epsilonIterate_logit)
    
    print("Mu:%.2f|Mu+:%.2f|Beta:%.2f|Sigma:%.2f|Epsilon:%.2f|EpIter:%.2f" % (mu_01, muIncr_01, beta_pos, sigma_pos, epsilon_pos, epsilonIterate_01))
    
    with HiddenPrints():
        f1=entropy_F1(beta=beta_pos,
                      epsilon=epsilon_pos,
                      epsilonIterate=epsilonIterate_01,
                      muIncr=muIncr_01,
                      sigma=sigma_pos,
                      mu=mu_01)
    
    return logit(f1)

In [16]:
entropy_F1_trx(beta_sp=inv_softplus(33),
               sigma_sp=inv_softplus(449),
               mu_logit=logit(0.5),
               muIncr_logit=logit(0.05),
               epsilon_sp=inv_softplus(31.8))

Mu:0.50|Mu+:0.05|Beta:33.00|Sigma:449.00|Epsilon:31.80|EpIter:0.00


-1.76332880889149

In [33]:
optimizer=None

In [34]:
pbounds = {'beta_sp': (inv_softplus(1), inv_softplus(500)), 
           'sigma_sp': (inv_softplus(1), inv_softplus(500)),
           'mu_logit': (logit(0.3), logit(0.8)),
           'muIncr_logit': (logit(0.01), logit(0.2)),
           'epsilon_sp': (inv_softplus(1), inv_softplus(500)),
           'epsilonIterate_logit': (logit(0), logit(0.25))}

In [35]:
optimizer = BayesianOptimization(
    f=entropy_F1_trx,
    pbounds=pbounds,
    random_state=1,
)

In [36]:
optimizer.probe(
    params={"beta_sp": inv_softplus(9), 
            "sigma_sp": inv_softplus(95),
            "mu_logit":logit(0.5),
            'epsilonIterate_logit': logit(0),
            "muIncr_logit":logit(0.1),
            "epsilon_sp":inv_softplus(4)},
    lazy=True,
)

In [37]:
optimizer.maximize(
    init_points=10,
    n_iter=50,
)

|   iter    |  target   |  beta_sp  | epsilo... | epsilo... | muIncr... | mu_logit  | sigma_sp  |
-------------------------------------------------------------------------------------------------
Mu:0.50|Mu+:0.10|Beta:9.00|Sigma:95.00|Epsilon:4.00|EpIter:0.00
|  1        |  2.546    |  9.0      | -124.3    |  3.982    | -2.197    |  0.0      |  95.0     |
Mu:0.37|Mu+:0.03|Beta:208.00|Sigma:46.00|Epsilon:1.04|EpIter:0.00
|  2        |  0.1301   |  208.8    | -35.57    |  0.5985   | -3.625    | -0.5195   |  46.66    |
Mu:0.52|Mu+:0.05|Beta:93.00|Sigma:342.00|Epsilon:198.71|EpIter:0.00
|  3        | -6.822    |  93.57    | -81.75    |  198.7    | -2.866    |  0.08901  |  342.8    |
Mu:0.52|Mu+:0.08|Beta:102.00|Sigma:279.00|Epsilon:14.22|EpIter:0.00
|  4        |  0.1935   |  102.7    | -16.12    |  14.22    | -2.444    |  0.08479  |  279.6    |
Mu:0.46|Mu+:0.18|Beta:70.00|Sigma:346.00|Epsilon:400.48|EpIter:0.00
|  5        | -7.249    |  70.66    | -99.93    |  400.5    | -1.488    | -0.1

|  36       |  1.489    |  49.62    | -74.88    |  474.3    | -2.689    |  1.26     |  158.4    |
Mu:0.36|Mu+:0.03|Beta:341.00|Sigma:377.00|Epsilon:7.12|EpIter:0.00
|  37       |  1.117    |  341.4    | -27.28    |  7.116    | -3.379    | -0.5867   |  377.7    |
Mu:0.57|Mu+:0.01|Beta:14.00|Sigma:100.00|Epsilon:8.72|EpIter:0.00
|  38       |  0.8306   |  14.04    | -84.71    |  8.715    | -4.23     |  0.2624   |  100.6    |
Mu:0.30|Mu+:0.05|Beta:55.00|Sigma:146.00|Epsilon:473.35|EpIter:0.00
|  39       | -7.428    |  55.49    | -83.78    |  473.4    | -2.871    | -0.8402   |  146.9    |
Mu:0.67|Mu+:0.04|Beta:15.00|Sigma:98.00|Epsilon:9.43|EpIter:0.00
|  40       |  1.324    |  15.97    | -85.3     |  9.434    | -3.209    |  0.7096   |  98.51    |
Mu:0.35|Mu+:0.05|Beta:337.00|Sigma:372.00|Epsilon:9.52|EpIter:0.00
|  41       |  0.5785   |  337.7    | -23.56    |  9.515    | -2.935    | -0.6149   |  372.6    |
Mu:0.67|Mu+:0.20|Beta:101.00|Sigma:271.00|Epsilon:13.31|EpIter:0.00
|  42      

In [38]:
optimizer.max

{'target': 2.545531312045612,
 'params': {'beta_sp': 8.999876582580297,
  'epsilonIterate_logit': -124.33959502167846,
  'epsilon_sp': 3.9815145531741134,
  'muIncr_logit': -2.197224577336219,
  'mu_logit': 0.0,
  'sigma_sp': 95.0}}

In [39]:
inv_logit(optimizer.max["target"])

0.92727273

In [40]:
params=optimizer.max["params"]

In [41]:
softplus(params["beta_sp"])

9.0

In [42]:
softplus(params["sigma_sp"])

95.0

In [43]:
inv_logit(params["mu_logit"])

0.5

In [44]:
inv_logit(params["muIncr_logit"])

0.10000000000000003

In [45]:
inv_logit(params["epsilonIterate_logit"])

0

In [46]:
softplus(params["epsilon_sp"])

4.0

In [48]:
entropy_F1(mu=0.5,
           muIncr=0.1,
           beta=9,
           sigma=95,
           epsilon=4,
           epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.5
Starting Iteration epsilon= 4
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 1012
NewMatrix Iterateblocks Total Time = 0.6923086130991578


>>>>>>>>>>>>
Starting Iteration mu= 0.6
Starting Iteration epsilon= 4
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 602
NewMatrix Iterateblocks Total Time = 0.568140247836709


>>>>>>>>>>>>
Starting Iteration mu= 0.7
Starting Iteration epsilon= 4
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 301
NewMatrix Iterateblocks Total Time = 0.3861687406897545


>>>>>>>>>>>>
Starting Iteration mu= 0.7999999999999999
Starting Iteration epsilon= 4
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 157
NewMatrix Iterateblocks Total Time = 0.3004871495068073


>>>>>>>>>>>>
Starting Iteration mu= 0.8999999999999999
Starting Iteration epsilon= 4
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 M

0.92727273